# [**Gpt3 API**](https://beta.openai.com/docs/api-reference/searches/create)

In [ ]:
!pip install openai

In [ ]:
import openai
# list api engines
openai.Engine.list()

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/Colab Notebooks/gpt3")
openai.api_key = open("gp.txt", "r").readline()

**prompt(s) to generate completions**

openai.Completion.create(
  parameters as args and kwargs
)
[parameters:](https://beta.openai.com/docs/api-reference/completions/create?lang=python)


In [4]:
result = openai.Completion.create(
  engine="text-davinci-001",   # select the engine
  temperature = 0.9,           # 0 makes the model deterministic and higher value close to 1 increases the randomness
  top_p = 0.1,                 # 0.1 means only the tokens comprising the top 10% probability mass are considered
  n = 1,                       # number of completions to generate for each prompt
  echo = True,                 # return back prompt along with completions 
  prompt="Hi bro, How are you?",   
  max_tokens= 10                # The maximum number of tokens to generate in the completion.
)

In [ ]:
result

In [6]:
result['choices'][0]['text']

"Hi bro, How are you?\n\nI'm good, thank you for asking"

# **Sentiment analysis**

**[Classification](https://beta.openai.com/docs/guides/classifications)**

In [7]:
train_data=[
    ["A happy moment", "Positive"],
    ["I am sad.", "Negative"],
    ["I am feeling awesome", "Positive"]
  ]
test_data = "It is a raining day :("
class_labels = ["Positive", "Negative", "Neutral"]

In [8]:
result = openai.Classification.create(
  search_model="ada",
  model="curie",
  examples = train_data,
  query = test_data,
  labels=class_labels,
)

In [9]:
result['label']

'Negative'

# [**Searches**](https://beta.openai.com/docs/api-reference/searches)

Given a query and a set of documents or labels, the model ranks each document based on its semantic similarity to the provided query.

In [17]:
# consider the following document in the format of jsonl
open("myfile.jsonl").readlines()

['{"text": "puppy A is happy", "metadata": "emotional state of puppy A"}\n',
 '{"text": "puppy B is sad", "metadata": "emotional state of puppy B"}']

In [15]:
openai.File.create(file=open("myfile.jsonl"), purpose="search") # upload the file

<File file id=file-M5OfOUchitlmffsUmcs40phw at 0x7fb73c758fb0> JSON: {
  "bytes": 139,
  "created_at": 1646491052,
  "filename": "myfile.jsonl",
  "id": "file-M5OfOUchitlmffsUmcs40phw",
  "object": "file",
  "purpose": "search",
  "status": "uploaded",
  "status_details": null
}

In [19]:
results = openai.Engine("ada").search(
    search_model="ada", 
    query="happy", 
    max_rerank=5,
    file="file-M5OfOUchitlmffsUmcs40phw"
)

In [24]:
results

<OpenAIObject list at 0x7fb73c712c50> JSON: {
  "data": [
    {
      "document": 0,
      "object": "search_result",
      "score": 584.489,
      "text": "puppy A is happy"
    }
  ],
  "model": "ada:2020-05-03",
  "object": "list"
}

In [27]:
print(results['data'][0]['text'])
print('similarity score =',results['data'][0]['score'])

puppy A is happy
similarity score = 584.489


**[Question answering](https://beta.openai.com/docs/guides/answers)**

The endpoint first searches over provided documents or file to find relevant context for the input question. Semantic search is used to rank documents by relevance to the question. The relevant context is combined with the provided examples and question to create the prompt for completion.

In [28]:
question = "which puppy is happy?"
documents = ["Puppy A is happy.", "Puppy B is sad."]
examples_context = "In 2017, U.S. life expectancy was 78.6 years."
examples = [["What is human life expectancy in the United States?","78 years."]]

In [29]:
results = openai.Answer.create(
  search_model="ada",
  model="curie",
  question = question,
  documents = documents,
  examples_context = examples_context,
  examples = examples,
  max_tokens = 5,
  stop=["\n", "<|endoftext|>"],
)

In [30]:
results

<OpenAIObject answer at 0x7fb73375cd70> JSON: {
  "answers": [
    "puppy A."
  ],
  "completion": "cmpl-4iQ9BYxW0ZfSv11tL8b9NAcJngk3j",
  "model": "curie:2020-05-03",
  "object": "answer",
  "search_model": "ada",
  "selected_documents": [
    {
      "document": 0,
      "text": "Puppy A is happy. "
    },
    {
      "document": 1,
      "text": "Puppy B is sad. "
    }
  ]
}

In [34]:
print(question)
print(results["answers"][0])

which puppy is happy?
puppy A.


In [3]:
openai.Completion.create(
  engine="davinci",
  prompt="Make a list of astronomical observatories:"
)

<OpenAIObject text_completion id=cmpl-4i78dw3Ag1G4RaB6GdR5IubuCMBXa at 0x7fd247dc3d70> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " Sary Shagan, Baykonur, La Palma, the Paran"
    }
  ],
  "created": 1646419219,
  "id": "cmpl-4i78dw3Ag1G4RaB6GdR5IubuCMBXa",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [9]:
prompt = """
Q: How can I reason using the first principles?
A: So long as you think and you believe in what you think, you are deceiving yourself. So let go of that belief. Reason from the first principles. Ask, why? Why? Ask yourself, Why are you doing that? Or, why did you do that? Question everything. Have a reason to explain every decision you make. Having reasons is important because a clear, definite reason is self-evident truth. It is right. It is good. It is perfect. Having reasons is important because it makes you think and be consistent, and because when you don’t have a reason you usually don’t think. Your reason is important because you are always aware of it, you will have the opportunity to question it to ensure it is true. If you have a reason to do something, then do it. If you don’t have a reason to do something, then don’t do it. Reason from first principles. Ask why, why, why
Q: Can you give me an example of how I can practise the first princliples thinking in my day to day life?
A: Question Everything
Q: Can you explain what's the best way to go about it?
A:
"""
response = openai.Completion.create(engine="davinci", prompt=prompt, stop="\n", temperature=0.9, max_tokens=721)
print(prompt.split(',')[-1].split('\n')[-2:][0])
print(u"{}".format(response.get('choices')[0].get('text')))

A:
"Where did all the questions come from? Well, the tone of my last post was clearly confrontational. I think it's fair, because this approach isn't for everyone. But the questions I was asking were part of a broader picture, and were meant to provoke thought -- not just to piss people off. I'd like to provide some context for where I'm coming from and how you can use questioning to be your own master."
